<a href="https://colab.research.google.com/github/Julita257/UMwF/blob/main/Kopia_notatnika_Time_series_NN_(keras)_example.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import pandas as pd
import yfinance as yf
import datetime as dt
import matplotlib.pyplot as plt
import math
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_absolute_error

In [ ]:
# importing libraries for learning
from keras.models import Sequential
from keras.layers import LSTM
from keras.layers import Dense
from keras.layers import SimpleRNN
from keras.layers import Dropout
from keras.layers import GRU, Bidirectional
from keras.optimizers import SGD
from sklearn import metrics
from sklearn.metrics import mean_squared_error

In [ ]:
#the start and end date
start_date = dt.datetime(2020,4,1)
end_date = dt.datetime(2024,4,1)

#loading from yahoo finance
data = yf.download("GOOGL",start_date, end_date)

print(data.head())
print(data.tail())


[*********************100%%**********************]  1 of 1 completed

                 Open       High        Low      Close  Adj Close    Volume
Date                                                                       
2020-04-01  56.200001  56.471001  54.674500  55.105000  55.105000  51970000
2020-04-02  55.000000  56.138500  54.656502  55.851501  55.851501  56410000
2020-04-03  55.735500  55.939499  53.754002  54.634998  54.634998  51374000
2020-04-06  56.650002  59.537498  56.250000  59.159500  59.159500  63320000
2020-04-07  60.850498  61.039001  58.862499  59.127998  59.127998  61620000
                  Open        High         Low       Close   Adj Close  \
Date                                                                     
2024-03-22  149.119995  151.580002  148.979996  150.770004  150.770004   
2024-03-25  149.940002  150.380005  147.820007  150.070007  150.070007   
2024-03-26  150.220001  152.259995  149.979996  150.669998  150.669998   
2024-03-27  151.179993  151.639999  148.899994  150.869995  150.869995   
2024-03-28  150.850006  

In [ ]:
# Setting 80 percent data for training
training_data = math.ceil(len(data) * .8)

#Splitting the dataset
train_data = data[:training_data_len].iloc[:,:1]
test_data = data[training_data_len:].iloc[:,:1]
print(train_data.shape, test_data.shape)

(804, 1) (201, 1)


In [ ]:
# Selecting Open Price values
dataset_train = train_data.Open.values
dataset_test = test_data.Open.values

# Reshaping 1D to 2D array
dataset_train = np.reshape(dataset_train, (-1,1))
dataset_test = np.reshape(dataset_test, (-1,1))

In [ ]:
scaler = MinMaxScaler(feature_range=(0,1))
# scaling dataset
scaled_train = scaler.fit_transform(dataset_train)
scaled_test = scaler.fit_transform(dataset_test)

In [ ]:
# The data is converted to Numpy array; one day ahead forecast
X_train = np.array(scaled_train[0:803])
y_train = np.array(scaled_train[1:804])

#Reshaping
X_train = np.reshape(X_train, (X_train.shape[0], X_train.shape[1],1))
y_train = np.reshape(y_train, (y_train.shape[0],1))
print("X_train :",X_train.shape,"y_train :",y_train.shape)

X_train : (803, 1, 1) y_train : (803, 1)


Siec rekurencyjna RNN

In [ ]:
# initializing the RNN
regressor = Sequential()

# adding RNN layers and dropout regularization
regressor.add(SimpleRNN(units = 50,
						activation = "tanh",
						return_sequences = True,
						input_shape = (X_train.shape[1],1)))
regressor.add(Dropout(0.2))

regressor.add(SimpleRNN(units = 50,
						activation = "tanh",
						return_sequences = True))

regressor.add(SimpleRNN(units = 50,
						activation = "tanh",
						return_sequences = True))

regressor.add( SimpleRNN(units = 50))

# adding the output layer
regressor.add(Dense(units = 1,activation='sigmoid'))

# compiling RNN
regressor.compile(optimizer = 'adam',
				loss = "mean_squared_error")

# fitting the model
regressor.fit(X_train, y_train,
              epochs = 20,
              batch_size = 2)
regressor.summary()


Epoch 1/20
402/402 [==============================] - 6s 6ms/step - loss: 0.0090
Epoch 2/20
402/402 [==============================] - 4s 10ms/step - loss: 0.0021
Epoch 3/20
402/402 [==============================] - 3s 8ms/step - loss: 0.0019
Epoch 4/20
402/402 [==============================] - 2s 4ms/step - loss: 0.0017
Epoch 5/20
402/402 [==============================] - 1s 4ms/step - loss: 0.0016
Epoch 6/20
402/402 [==============================] - 2s 4ms/step - loss: 0.0015
Epoch 7/20
402/402 [==============================] - 2s 5ms/step - loss: 0.0017
Epoch 8/20
402/402 [==============================] - 3s 6ms/step - loss: 0.0016
Epoch 9/20
402/402 [==============================] - 2s 5ms/step - loss: 0.0015
Epoch 10/20
402/402 [==============================] - 2s 4ms/step - loss: 0.0015
Epoch 11/20
402/402 [==============================] - 2s 4ms/step - loss: 0.0016
Epoch 12/20
402/402 [==============================] - 2s 4ms/step - loss: 0.0016
Epoch 13/20
402/402 [===

Sieć typu LSTM


In [ ]:
#Initializing the RNN
regressorLSTM = Sequential()
regressorLSTM.add(LSTM(units = 50,
                   return_sequences = True,
                   input_shape = (X_train.shape[1],1)))
regressorLSTM.add(Dropout(0.2))
# Adding second hidden layer
regressorLSTM.add(LSTM(units = 50,
                   return_sequences = True))
regressorLSTM.add(Dropout(0.2))
# Adding dense layer to get the final output. The input of n-1 layer, will be the output for n layer.
regressorLSTM.add(Dense(units = 1))

regressorLSTM.compile(optimizer = 'adam',
				loss = "mean_squared_error")

# fitting the model
regressorLSTM.fit(X_train, y_train,
              epochs = 20,
              batch_size = 2)
regressorLSTM.summary()

Epoch 1/20
402/402 [==============================] - 7s 6ms/step - loss: 0.0445
Epoch 2/20
402/402 [==============================] - 2s 5ms/step - loss: 0.0048
Epoch 3/20
402/402 [==============================] - 2s 5ms/step - loss: 0.0040
Epoch 4/20
402/402 [==============================] - 2s 5ms/step - loss: 0.0037
Epoch 5/20
402/402 [==============================] - 2s 4ms/step - loss: 0.0035
Epoch 6/20
402/402 [==============================] - 2s 4ms/step - loss: 0.0029
Epoch 7/20
402/402 [==============================] - 2s 6ms/step - loss: 0.0028
Epoch 8/20
402/402 [==============================] - 2s 5ms/step - loss: 0.0024
Epoch 9/20
402/402 [==============================] - 2s 4ms/step - loss: 0.0022
Epoch 10/20
402/402 [==============================] - 2s 5ms/step - loss: 0.0021
Epoch 11/20
402/402 [==============================] - 2s 4ms/step - loss: 0.0022
Epoch 12/20
402/402 [==============================] - 2s 4ms/step - loss: 0.0016
Epoch 13/20
402/402 [====

Porównanie modeli

In [ ]:
# The data is converted to Numpy array; one day ahead forecast
X_test = np.array(scaled_test[0:200])
y_test = np.array(scaled_test[1:201])

#Reshaping
X_test = np.reshape(X_test, (X_test.shape[0], X_test.shape[1],1))
y_test = np.reshape(y_test, (y_test.shape[0],1))
print("X_test :",X_test.shape,"y_test :",y_test.shape)

y_RNN = regressor.predict(X_test)
y_LSTM = regressorLSTM.predict(X_test)
y_LSTM = np.reshape(y_LSTM, (y_LSTM.shape[0],1))

y_RNN_O = scaler.inverse_transform(y_RNN)
y_LSTM_O = scaler.inverse_transform(y_LSTM)

print(mean_absolute_error(scaler.inverse_transform(y_test), y_RNN_O))
print(mean_absolute_error(scaler.inverse_transform(y_test), y_LSTM_O))

X_test : (200, 1, 1) y_test : (200, 1)
7/7 [==============================] - 0s 3ms/step
1.8477799606323242
1.6755125427246094


TODO:
1. dodaj sieć typu GRU oraz model addytywny, porównaj wyniki
2. przedstaw wyniki na wykresach szeregów czasowych
3. wykonaj hiperparametryzacje wybranego modelu